In [72]:
import yaml
import pprint

slot_dict = yaml.safe_load(open("intent_slots_v3.yml", 'r'))
pprint.pprint(slot_dict)

{'Intent-Slot': {'Bye': [],
                 'Crop Disease': ['Crop Type',
                                  'User Location',
                                  'Crop Symptoms',
                                  'Solution'],
                 'Crop Knowledge': ['Crop Type', 'User Location', 'Solution'],
                 'Crop Quality Query': ['Crop Type',
                                        'User Location',
                                        'Solution'],
                 'Farming Tips': ['Crop Type', 'Solution'],
                 'Fertilizer Recommendation': ['Crop Type',
                                               'User Location',
                                               'Solution'],
                 'Greeting': [],
                 'Machinery Related': ['Machine Problems', 'Solution'],
                 'Seed Availability': ['Crop Type',
                                       'User Location',
                                       'Solution'],
                 'Seed Re

In [70]:
from collections import defaultdict
from random import randrange

class IntentClassifier:
    def __init__(self):
        self.intent_keywords_dict = {
                                        "Greeting": ['aslam alaikum','aoa'],
                                        "Bye": ['allah hafiz','bye'],
                                        "Crop Disease": ['solution','disease','bemari','masla','gira','hal','jala','mur','hamla'],
                                        "Crop Quality": ['paidawaar','behtar','bach'],
                                        "Yielding Season": ['kasht','kab','date','month','mahinay','mausam','saal','waqt','kb'],
                                        "Seed Recommendation": ['recommend','kasht','fasal'],
                                        "Machinery Related": ['machine','drill','tractor'],
                                        "Seed Availibility": ['available','milayga'],
                                        "Spray Recommendation": ['spray'],
                                        "Fertilizer Recommendation": ['fertilizer','khad'],
                                        "Crop Knowledge": ['fasal','kon','tareeka'],
                                        "Farming Tips":['guidance']
                                    }
        
    def classify_intent(self,farmer_input):
        farmer_input_lower = farmer_input.lower()
        
        intent_list = []
        for key,value in self.intent_keywords_dict.items():
            for v in value:
                if v in farmer_input_lower:
                    intent_list.append(key)
                    break
        if len(intent_list)==0:
            intent_list.append("UNK")
        return intent_list
    
class EntityExtractor:
    def __init__(self):
        self.entity_keywords_dict = {
                                        "Crop Type": ['gandum','chawal','monji','barseen','canola','bajra','sun flower','Shalgam','Canola','Bhindi','makai','wheat','shimla mirch','tamatar','rice','kino','kapaas','sugarcane','napire grass','khjoor'],
                                        "User Location": ['barani','jhelum','punjab','sindh'],
                                        "Disease Symptoms": ['keera','safaid'],
                                        "Machine Problems":['tractor'],
                                    }
    def extract_entities(self,farmer_input):
        farmer_input_lower = farmer_input.lower()
        
        entity_dict = defaultdict(list)
        for key,value in self.entity_keywords_dict.items():
            for v in value:
                if v in farmer_input_lower:
                    entity_dict[key].append(v)
        return entity_dict

class KisaanDost:
    def __init__(self):
        self.chat_history = []
        self.problems = {'Solved':[],'Unsolved':[]}
        self.intent_classifier = IntentClassifier()
        self.entity_extractor = EntityExtractor()
        self.knowledge_base = None
    def query_knowledge_base(self,query_info):
        pass
    def get_random_slot_prompt(self,slot):
        prompts = slot_dict['Slot-Prompt'][slot]
        return prompts[randrange(len(prompts))]
    def get_solution(self,problem):
        return self.get_random_slot_prompt('Solution')
    def classify_intent(self,user_input_dict):
        """
        Classifies message into one or more intents.
        If no intent is detected in the message, UNK intent is classified. This is the only
        situation in which UNK intent is classified. UNK stands for 'unknown'.
        """
        user_input_dict['Intent'] = self.intent_classifier.classify_intent(user_input_dict['Message'])
    def extract_entities(self,user_input_dict):
        user_input_dict['Entities'] = self.entity_extractor.extract_entities(user_input_dict['Message'])
        for problem in self.problems['Unsolved']:
            for slot_key in problem['Slots'].keys():
                if slot_key in user_input_dict['Entities']:
                    problem['Slots'][slot_key] = user_input_dict['Entities'][slot_key]
    def detect_problem(self,user_input_dict):
        for i in user_input_dict['Intent']:
            if i in ['Greeting','Bye','UNK']:
                continue
            problem = i
            slots = {k:None for k in slot_dict['Intent-Slot'][i]}
            self.problems['Unsolved'].append({'Problem':problem,
                                              'Slots':slots,
                                              'Solution':None})
    def get_user_input(self,user_input=None):
        if not user_input:
            user_input = input("USER: ")
        self.chat_history.append({'Agent':'User',
                                  'Message':user_input,
                                  'Intent':["UNK"],
                                  'Entities':[]})
    def generate_response(self,user_input_dict=None):
        try:
            if not user_input_dict:
                self.chat_history.append({'Agent':'KisaanBot',
                                          'Message':'assalam alaikum'})
                return
            if 'Bye' in user_input_dict['Intent']:
                self.chat_history.append({'Agent':'KisaanBot',
                                          'Message':'allah hafiz'})
                return
            if 'UNK' in user_input_dict['Intent'] and len(user_input_dict['Entities'])==0:
                self.chat_history.append({'Agent':'KisaanBot',
                                          'Message':'meje ap ke baat samajh nhai ai'})
                return
            message = ''  
            if 'Greeting' in user_input_dict['Intent']:
                message += 'walaikum assalam '
            if len(self.problems['Unsolved'])==0:
                message += 'ap ka kia masla hai'
                self.chat_history.append({'Agent':'KisaanBot',
                                          'Message':message})  
                return
            current_problem = self.problems['Unsolved'][0]
            all_slots_filled = True
            for slot_key in current_problem['Slots'].keys():
                if not current_problem['Slots'][slot_key]:
                    message += self.get_random_slot_prompt(slot_key)
                    all_slots_filled = False
                    break
            if all_slots_filled:
                solution = get_solution(current_problem)
                message += solution
                current_problem['Solution'] = solution
                problem_index = self.problems['Unsolved'].index(current_problem)
                solved_problem = self.problems['Unsolved'].pop(problem_index)
                self.problems['Solved'].append(solved_problem)
            self.chat_history.append({'Agent':'KisaanBot',
                                      'Message':message})        
        except Exception as e:
            self.chat_history.append({'Agent':'KisaanBot',
                                      'Message':'mein abhi chal nhai raha baad mein koshish kejiye ga'}) 
    def get_user_voice_input(self):
        pass
    def generate_voice_response(self):
        pass
    def return_last_response_message(self):
        for c in self.chat_history[::-1]:
            if c['Agent']=='KisaanBot':
                return c['Message']
    def run_chatbot(self):
        while(1):
            self.get_user_input()
            if self.chat_history[-1]['Message'] == 'q':
                break                
            self.classify_intent(self.chat_history[-1])
            self.detect_problem(self.chat_history[-1])
            self.extract_entities(self.chat_history[-1])
            print("-----------------")
            if len(self.chat_history[-1]['Intent'])>0:
                print("Intents Detected:",self.chat_history[-1]['Intent'])
            if len(self.chat_history[-1]['Entities'])>0:
                print("Entities Detected:",dict(self.chat_history[-1]['Entities']))
            print("-----------------")
            self.generate_response(self.chat_history[-1])
            print(f"{self.chat_history[-1]['Agent']}: {self.chat_history[-1]['Message']}")
            print()

In [75]:
myKisaanDost = KisaanDost()
myKisaanDost.run_chatbot()

USER: aslam alaikum
-----------------
Intents Detected: ['Greeting']
-----------------
KisaanBot: walaikum assalam ap ka kia masla hai

USER: barani ilaky k liye double coulter drill kesee rahy gee me iss dfa usky sath gandum kaasht krna chahta huu agr koi istemal kr rha hy tu mshwra de
-----------------
Intents Detected: ['Machinery Related']
Entities Detected: {'Crop Type': ['gandum'], 'User Location': ['barani']}
-----------------
KisaanBot: ap ke machinery ka kia issue hai

USER: tractor
-----------------
Intents Detected: ['Machinery Related']
Entities Detected: {'Machine Problems': ['tractor']}
-----------------
KisaanBot: phir ______ hai ap ka hal

USER: q
